In [1]:
import json
import pydgraph

from pydgraph import DgraphClient, DgraphClientStub
from grapl_analyzerlib.schemas import *

from grapl_analyzerlib.schemas.schema_builder import ManyToMany

In [2]:
def set_schema(client, schema, engagement=False):
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
                 
                
def drop_all(client):
    op = pydgraph.Operation(drop_all=True)
    client.alter(op)
    
def format_schemas(schema_defs):
    schemas = "\n\n".join([schema.to_schema_str() for schema in schema_defs])

    types = "\n\n".join([schema.generate_type() for schema in schema_defs])

    return "\n".join([
        "  # Type Definitions",
        types,
        "\n  # Schema Definitions",
        schemas,
    ])

In [3]:
mclient = DgraphClient(DgraphClientStub('alpha0.mastergraphcluster.grapl:9080'))
eclient = DgraphClient(DgraphClientStub('alpha0.engagementgraphcluster.grapl:9080'))

In [8]:
# drop_all(mclient)
# drop_all(eclient)

schemas = (
    AssetSchema(),
    ProcessSchema(),
    FileSchema(),
    IpConnectionSchema(),
    IpAddressSchema(),
    IpPortSchema(),
    NetworkConnectionSchema(),
    ProcessInboundConnectionSchema(),
    ProcessOutboundConnectionSchema(),  
)

schema_str = format_schemas(schemas)
# print(schema_str)
set_schema(mclient, schema_str)


risk_schema = RiskSchema()
lens_schema = LensSchema()
eg_schemas = [s.with_forward_edge('risks', ManyToMany(RiskSchema), 'risky_nodes') for s in schemas]
eg_schemas.extend([risk_schema, lens_schema])
eg_schema_str = format_schemas(eg_schemas)
set_schema(eclient, eg_schema_str)

  # Type Definitions
  type Asset {
      node_key: string
      hostname: string
      risks: uid  # type: Risk
  }


  type Process {
      node_key: string
      asset_id: string
      image_name: string
      process_name: string
      arguments: string
      process_id: int
      created_timestamp: int
      terminate_time: int
      children: uid  # type: Process
      bin_file: uid  # type: File
      created_files: uid  # type: File
      deleted_files: uid  # type: File
      read_files: uid  # type: File
      wrote_files: uid  # type: File
      created_connections: uid  # type: ProcessOutboundConnection
      inbound_connections: uid  # type: ProcessInboundConnection
      risks: uid  # type: Risk
  }


  type File {
      node_key: string
      file_name: string
      asset_id: string
      file_path: string
      file_extension: string
      file_mime_type: string
      file_version: string
      file_description: string
      file_product: string
      file_company: stri

In [5]:
import os
import string

from hashlib import pbkdf2_hmac, sha256
from random import randint, choice

import boto3

def hash_password(cleartext, salt) -> str:
    hashed = sha256(cleartext).digest()
    return pbkdf2_hmac(
        'sha256', 
        hashed,
        salt,
        512000
    ).hex()

def create_user(username, cleartext):
    assert cleartext
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('user_auth_table')
    
    # We hash before calling 'hashed_password' because the frontend will also perform
    # client side hashing
    cleartext += "f1dafbdcab924862a198deaa5b6bae29aef7f2a442f841da975f1c515529d254";
    
    cleartext += username;

    hashed = sha256(cleartext.encode('utf8')).hexdigest()
    
    for i in range(0, 5000):
        hashed = sha256(hashed.encode('utf8')).hexdigest()
    
    salt = os.urandom(16)
    password = hash_password(hashed.encode('utf8'), salt)
    table.put_item(
        Item={
            'username': username,
            'salt': salt,
            'password': password
        }
    )
    
    

allchar = string.ascii_letters + string.punctuation + string.digits
password = "".join(choice(allchar) for x in range(randint(14, 16)))
print(f'your password is {password}')
username = ''
assert username, 'Replace the username with your desired username'
create_user(username, password)
password = ""
print("""Make sure to clear this cell and restart the notebook to ensure your password does not leak!""")

SyntaxError: invalid syntax (<ipython-input-5-3ec0c580d450>, line 23)

In [11]:
# CLEAR CACHE
def clear_redis_caches():
    from redis import Redis

    def chunker(seq, size):
        return [seq[pos:pos + size] for pos in range(0, len(seq), size)]

    CACHE_ADDRS = [
    
    ]
    CACHE_PORT = 6379
    for CACHE_ADDR in CACHE_ADDRS:
        r = Redis(host=CACHE_ADDR, port=CACHE_PORT, db=0, decode_responses=True)

        for keys in chunker([k for k in r.keys()], 10000):
            r.delete(*keys)
            
clear_redis_caches()
